# 🧬 RNA SSP Tutorial 3/4: Model Training - Teaching the Model to Predict

Welcome to the third tutorial in our series. So far, we have:
1. Prepared our dataset ([01_data_preparation.ipynb](01_data_preparation.ipynb))
2. Initialized our model ([02_model_initialization.ipynb](02_model_initialization.ipynb))

Now we have all the ingredients ready. In this tutorial, we will combine them to **train** the model. Training is the process of teaching the model to make accurate predictions by showing it examples from our dataset and adjusting its internal parameters.

This tutorial will cover:
1. **The Concept of Supervised Training**: What it means to train a model
2. **Trainers in OmniGenBench**: Different training engines available
3. **The Training Process**: Step-by-step configuration and execution
4. **Evaluation and Results**: Measuring performance

By the end of this tutorial, you will have a fine-tuned model saved on your disk, ready for inference.

### 1. The Concept of Supervised Training

At its heart, **supervised training** is like teaching a student with a textbook and an answer key. We show the model an input (an RNA sequence), let it make a prediction, and then compare its prediction to the correct answer (the structure labels). The difference is quantified by a **loss function**. The goal is to minimize this loss.

This process is iterative and happens in a **training loop**, which consists of four main steps:

```mermaid
graph LR
    A["1. Forward Pass<br/>(Make Prediction)"] --> B["2. Loss Calculation<br/>(Measure Error)"]
    B --> C["3. Backward Pass<br/>(Compute Gradients)"]
    C --> D["4. Optimizer Step<br/>(Update Weights)"]
    D --> A
    
    style A fill:#e1f5fe
    style B fill:#f3e5f5
    style C fill:#e8f5e8
    style D fill:#fff3e0
```

To manage this entire process, we need:
- A **Model** to be trained
- **DataLoaders** to supply batches of data
- A **Loss Function** to measure error (Cross-Entropy for classification)
- An **Optimizer** to update weights (typically AdamW)
- **Evaluation Metrics** (F1-score, Accuracy) to assess performance

A **Trainer** encapsulates this entire training loop, providing a clean interface.

### 2. Trainers in OmniGenBench

`OmniGenBench` provides flexible training options:

| Trainer Class | Key Feature | When to Use |
| --- | --- | --- |
| `Trainer` | Native PyTorch | Simple, single-GPU training |
| **`AccelerateTrainer`** | **Distributed via `accelerate`** | **Recommended - Scales from single to multi-GPU** |
| `HFTrainer` | Hugging Face integration | If using HF ecosystem heavily |

For this tutorial, we use **`AccelerateTrainer`** - it's powerful, scalable, and easy to use.

## 🛠️ The Training Process in Action

Let's put everything together and train our model!

### Step 1: Environment Setup and Imports

In [ ]:
# Install if needed
# !pip install omnigenbench -U

In [ ]:
import torch
from omnigenbench import (
    OmniTokenizer,
    OmniModelForTokenClassification,
    OmniDatasetForTokenClassification,
    ClassificationMetric,
    AccelerateTrainer,
)

print("✅ Libraries imported successfully!")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🎯 CUDA available: {torch.cuda.is_available()}")

### Step 2: Configuration

In [ ]:
# Model and dataset configuration
model_name_or_path = "yangheng/OmniGenome-52M"
dataset_name = "RNA-SSP-Archive2"

# Label mapping
label2id = {"(": 0, ")": 1, ".": 2}
id2label = {v: k for k, v in label2id.items()}

# Training parameters
max_length = 512
batch_size = 8
num_epochs = 3
learning_rate = 2e-5
max_examples = 1000  # For quick testing; set to None for full dataset

# Output directory
output_dir = "ogb_rna_structure_finetuned"

print("✅ Configuration complete!")
print(f"📊 Model: {model_name_or_path}")
print(f"📊 Dataset: {dataset_name}")
print(f"📊 Batch size: {batch_size}")
print(f"📊 Epochs: {num_epochs}")
print(f"📊 Learning rate: {learning_rate}")

### Step 3: Load Data

We'll reuse the data loading code from Tutorial 1.

In [ ]:
# Initialize tokenizer
tokenizer = OmniTokenizer.from_pretrained(model_name_or_path)
print(f"✅ Tokenizer loaded: {model_name_or_path}")

# Load datasets
print("🏗️ Loading datasets...")
datasets = OmniDatasetForTokenClassification.from_hub(
    dataset_name_or_path=dataset_name,
    tokenizer=tokenizer,
    max_length=max_length,
    label2id=label2id,
    max_examples=max_examples,
)

print(f"📊 Datasets loaded: {list(datasets.keys())}")
for split, dataset in datasets.items():
    print(f"  - {split}: {len(dataset)} samples")

### Step 4: Initialize Model

We'll reuse the model initialization from Tutorial 2.

In [ ]:
# Initialize model
model = OmniModelForTokenClassification(
    model_name_or_path,
    tokenizer=tokenizer,
    label2id=label2id,
    id2label=id2label,
)

print(f"✅ Model initialized: {model_name_or_path}")
print(f"📊 Total parameters: {sum(p.numel() for p in model.parameters()):,}")

### Step 5: Define Evaluation Metrics

We'll use F1-score and Accuracy to evaluate our model's performance. The `ignore_y=-100` parameter tells the metric to ignore special tokens.

In [ ]:
# Define evaluation metrics
metric_functions = [
    ClassificationMetric(ignore_y=-100).f1_score,
    ClassificationMetric(ignore_y=-100).accuracy_score,
]

print("✅ Evaluation metrics configured:")
print("  - F1-score (macro-averaged)")
print("  - Accuracy")
print("  - Special tokens (label=-100) will be ignored")

### Step 6: Configure and Run Trainer

Now we configure the `AccelerateTrainer` with all our components and launch training!

In [ ]:
# Initialize trainer
trainer = AccelerateTrainer(
    model=model,
    train_dataset=datasets["train"],
    eval_dataset=datasets["valid"],
    test_dataset=datasets["test"],
    compute_metrics=metric_functions,
    # Training parameters
    batch_size=batch_size,
    num_epochs=num_epochs,
    learning_rate=learning_rate,
    # Optimization settings
    warmup_ratio=0.1,
    weight_decay=0.01,
    # Logging and checkpointing
    logging_steps=10,
    eval_steps=50,
    save_steps=100,
    output_dir=output_dir,
)

print("✅ Trainer configured!")
print(f"📊 Training will run for {num_epochs} epochs")
print(f"📊 Model checkpoints will be saved to: {output_dir}")

### Step 7: Start Training

This is where the magic happens! The trainer will:
1. Run the training loop for the specified number of epochs
2. Periodically evaluate on the validation set
3. Save checkpoints
4. Log metrics

**Note**: This may take several minutes to hours depending on your hardware and dataset size.

In [ ]:
print("🎓 Starting training...")
print("=" * 60)

# Train the model
metrics = trainer.train()

print("=" * 60)
print("✅ Training completed!")

### Step 8: Review Training Results

Let's examine the final metrics.

In [ ]:
print("📊 Final Training Metrics:")
print("=" * 60)
for key, value in metrics.items():
    print(f"  {key}: {value}")
print("=" * 60)

### Step 9: Save the Model

Save the trained model for later use.

In [ ]:
# Save the trained model
trainer.save_model(output_dir)

print(f"✅ Model saved to: {output_dir}")
print("\n📦 Saved files:")
import os
if os.path.exists(output_dir):
    for file in os.listdir(output_dir):
        print(f"  - {file}")

### Step 10: Test Set Evaluation (Optional)

Evaluate the trained model on the test set to get final performance metrics.

In [ ]:
if "test" in datasets:
    print("🧪 Evaluating on test set...")
    test_metrics = trainer.evaluate(datasets["test"])
    
    print("\n📊 Test Set Performance:")
    print("=" * 60)
    for key, value in test_metrics.items():
        print(f"  {key}: {value}")
    print("=" * 60)
else:
    print("⚠️ No test set available for evaluation")

## 📊 Understanding the Metrics

### F1-Score
- Harmonic mean of precision and recall
- Range: 0.0 (worst) to 1.0 (best)
- Good for imbalanced classes (more `)` and `(` than `.`)

### Accuracy
- Percentage of correctly predicted tokens
- Range: 0.0 (worst) to 1.0 (best)
- Easy to interpret but can be misleading with imbalanced data

### What are Good Scores?
For RNA secondary structure prediction:
- **F1 > 0.80**: Excellent
- **F1 > 0.70**: Good
- **F1 > 0.60**: Acceptable
- **F1 < 0.60**: Needs improvement

## 💡 Training Tips and Tricks

### Improving Performance

1. **More Data**: Use full dataset (set `max_examples=None`)
2. **More Epochs**: Train for 5-10 epochs instead of 3
3. **Larger Model**: Try OmniGenome-186M or 418M
4. **Learning Rate**: Experiment with 1e-5 to 5e-5
5. **Batch Size**: Increase if you have GPU memory

### Common Issues

| Issue | Solution |
|-------|----------|
| **Out of Memory** | Reduce batch_size or max_length |
| **Training Too Slow** | Increase batch_size or use larger GPU |
| **Overfitting** | Reduce num_epochs or add dropout |
| **Underfitting** | Train longer or use larger model |
| **Unstable Loss** | Reduce learning_rate |

## 📚 Summary and Next Steps

In this tutorial, we:
1. ✅ Understood the concept of supervised training
2. ✅ Explored OmniGenBench trainer options
3. ✅ Configured AccelerateTrainer with all components
4. ✅ Trained the model on RNA structure data
5. ✅ Evaluated performance with F1-score and accuracy
6. ✅ Saved the trained model for future use

### What We've Accomplished
```python
# Complete training in ~15 lines!
trainer = AccelerateTrainer(
    model=model,
    train_dataset=datasets["train"],
    eval_dataset=datasets["valid"],
    compute_metrics=metric_functions,
)
metrics = trainer.train()
trainer.save_model(output_dir)
```

### Key Takeaways
- **AccelerateTrainer** handles the entire training loop
- **Metrics** help us understand model performance
- **Checkpointing** allows us to resume training
- **Evaluation** on validation set prevents overfitting

### Next: Model Inference
Now that we have a trained model, proceed to **[04_model_inference.ipynb](04_model_inference.ipynb)** to:
- Load the trained model
- Make predictions on new RNA sequences
- Validate predicted structures
- Interpret and visualize results

Time to put our model to work! 🚀